<a href="https://colab.research.google.com/github/mayank-raj1/Fine-tuned-Phi3Mini/blob/main/FineTune_Phi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine Tunning Phi-3-mini using DPO


## Introduction
The goal of this notebook is to fine tune Phi-3 mini model using Direct Preference Optimization(DPO) from an synthetic data set, making the model output richer information without explicitly guidance prompt during inference, leding to drastic token savings.


## Let's Being!

### Importing Packages


In [ ]:
!pip install datasets trl peft bitsandbytes wandb accelerate transformers

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import os
import gc
import torch
import transformers
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    BitsAndBytesConfig
)
from datasets import load_dataset
from peft import (
    LoraConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from trl import DPOTrainer, setup_chat_format
import bitsandbytes as bnb

### Loading the model and tokenizer

In [ ]:
base_model = "microsoft/Phi-3-mini-4k-instruct"
new_model = "MayankDPOPhi-3--Mini"

In [ ]:
# The bits and bytes config we use for quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
    device_map="auto",
)
# Reference model
ref_model = AutoModelForCausalLM.from_pretrained(
    base_model,
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
    device_map="auto",
)

In [ ]:
# LoRA configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['k_proj', 'gate_proj', 'v_proj', 'up_proj', 'q_proj', 'o_proj', 'down_proj']
)

### Loading and formating Data Sets


In [ ]:
# Load dataset
dataset = load_dataset("Intel/orca_dpo_pairs")['train']

In [ ]:
dataset = dataset.shuffle(seed=42).select(range(1000))

In [ ]:
!wget -L https://raw.githubusercontent.com/chujiezheng/chat_templates/main/chat_templates/phi-3.jinja

In [ ]:
chat_template = open('phi-3.jinja').read()
chat_template = chat_template.replace('    ', '').replace('\n', '')
tokenizer.chat_template = chat_template

In [ ]:
def dataset_format(example):
    # Format system
    if len(example['system']) > 0:
        message = {"role": "system", "content": example['system']}
        system = tokenizer.apply_chat_template([message], tokenize=False)
    else:
        system = ""
    # Format instruction
    message = {"role": "user", "content": example['question']}
    prompt = tokenizer.apply_chat_template([message], tokenize=False, add_generation_prompt=True)
    # Format chosen answer
    chosen = example['chosen'] + "<|end|>\n"
    # Format rejected answer
    rejected = example['rejected'] + "<|end|>\n"
    return {
        "prompt": system + prompt,
        "chosen": chosen,
        "rejected": rejected,
    }

In [ ]:
original_columns = dataset.column_names
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

dataset = dataset.map(
    dataset_format,
    remove_columns=original_columns,
    num_proc= os.cpu_count(),
)

In [ ]:
# notice the specific llama3 tags like <|eot_id|> which show that the chat template formatting worked
dataset[19]

In [ ]:
# Add the new fromated data set

### Creating the DPO Trainer

In [ ]:
import wandb

wandb.login()

In [ ]:
training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    learning_rate=5e-5,
    lr_scheduler_type="cosine",
    max_steps=160,
    save_strategy="no",
    logging_steps=1,
    output_dir=new_model,
    optim="paged_adamw_32bit",
    warmup_steps=70,
    bf16=True,
    report_to="wandb",
)

In [ ]:
dpo_trainer = DPOTrainer(
    model,
    ref_model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    peft_config=peft_config,
    beta=0.1,
    max_prompt_length=512,
    max_length=1024,
    force_use_ref_model=True
)

In [ ]:
!nvidia-smi

In [ ]:
# Fine-tune model with DPO
dpo_trainer.train()

### Saving and testing model

In [ ]:
dpo_trainer.model.save_pretrained("final_ckpt")

In [ ]:
tokenizer.save_pretrained("final_ckpt")

In [ ]:
# Flush memory
del dpo_trainer, model, ref_model
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# Reload model in FP16 (instead of NF4)
base_model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    return_dict=True,
    torch_dtype=torch.float16,
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
tokenizer.chat_template = chat_template

In [ ]:
# Merge base model with the adapter
model = PeftModel.from_pretrained(base_model, "final_ckpt")
model = model.merge_and_unload()

# Save model and tokenizer
model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Create pipeline
pipeline = transformers.pipeline(
    "text-generation",
    model=new_model,
    tokenizer=tokenizer
)

In [ ]:
# Format prompt
message = [
    {"role": "system", "content": "You are a helpful assistant chatbot that provides concise answers."},
    {"role": "user", "content": "What are GPUs and why would I use them for machine learning tasks?"}
]
tokenizer = AutoTokenizer.from_pretrained(new_model)
prompt = tokenizer.apply_chat_template(message, add_generation_prompt=True, tokenize=False)

# Generate text
sequences = pipeline(
    prompt,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    num_return_sequences=1,
    max_length=200,
)
print(sequences[0]['generated_text'])